In [1]:
import pickle
import time
import os, glob, sys

import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage.io import imread, imshow, imsave

import h5py

%matplotlib inline

from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler

import functools
#from eval import *

import tensorflow as tf
import random
%env CITYSCAPES_DATASET = /home/rvygon/data
from tensorflow.metrics import *
%load_ext autoreload
%autoreload 2
LETTERS = list('abcdefghjklmnop')
NUMBERS = [i for i in range(1, 16)]
print(len(LETTERS))
print(len(NUMBERS))
DATA_FILE = 'data.csv'

/Users/sergevkim/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


env: CITYSCAPES_DATASET=/home/rvygon/data
15
15


In [2]:
class Position:
    def __init__(self, turn, seq, label=None):    # seq is a list of cells (x, y)
        self.fir = np.array([[0 for j in range(15)] for i in range(15)])
        self.sec = np.array([[0 for j in range(15)] for i in range(15)])
        self.thi = np.array([[turn for j in range(15)] for i in range(15)])
        self.label = label
        for i in range(len(seq)):
            if i % 2 == 0:
                self.fir[seq[i][0]][seq[i][1]] = 1
            else:
                self.sec[seq[i][0]][seq[i][1]] = -1
        self.image = np.array([self.fir, self.sec, self.thi])
        self.object = np.array([self.image, self.label])

def translate(coords):    # type(coords) == str: k8, h1, etc. Want tuple
    coordFir = LETTERS.index(coords[0])
    coordSec = int(coords[1:]) - 1
    return (coordSec, coordFir)

def translate2(coords):   #type(coords) == tuple: (7, 7), (9, 9), etc. Want a vector of nums for each class
    res = coords[0] * 15 + coords[1]
    return res
    

def to_seq(game):
    seq = []
    if game[0] == 'black':
        winner = 1
    else:
        winner = -1
    for i in range(1, len(game) - 1):
        seq.append(translate(game[i]))
    return winner, seq

allPositions = []

reader = open('train-1.renju', 'r')
for i in range(5000):
    a = list(map(str, reader.readline().split()))
    winner, seq = to_seq(a)
    if winner == 1:
        for j in range(0, len(seq) - 2, 2):
            seq2 = seq[:j]
            cur = translate2(seq[j])
            if len(seq2) % 2 == 0:
                player = 1
            else:
                player = -1
            p = Position(player, seq2, cur)
            allPositions.append(p.object)
    else:
        for j in range(1, len(seq) - 2, 2):
            seq2 = seq[:j]
            cur = translate2(seq[j])
            if len(seq2) % 2 == 0:
                player = 1
            else:
                player = -1
            p = Position(player, seq2, cur)
            allPositions.append(p.object)

#print(allPositions)
#allPositions = np.array(allPositions)
with open("data.txt", "wb") as fp:   #Pickling
    pickle.dump(allPositions, fp)
print('Done!')
with open("data.txt", "rb") as fp:   # Unpickling
    allP = pickle.load(fp)
print(len(allP))
print(allP[1][1])

Done!
72393
98
